In [1]:
from ipynb.fs.full.aoc_helpers import *

In [2]:
import re
import numpy as np
import math
import itertools as it

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, partial
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

In [3]:
Input = partial(Input, year=2016)

## [Day 20](http://adventofcode.com/2016/day/20): Firewall Rules

Given a list of blocked IP ranges, what is the first IP which is not blocked?

### Example
#### Input
5-8  
0-2  
4-7  
#### Output
3

In [4]:
def parse(line):
    match = re.match(r'(\d+)-(\d+)', line)
    return (int(match[1]), int(match[2]))

def unblocked(ips):
    matches = [parse(line) for line in ips]
    highest = 0
    for (lo, hi) in sorted(matches):
        yield from range(highest+1, lo)
        highest = max(highest, hi)

In [5]:
next(unblocked(Input(20)))

4793564

In [6]:
len(list(unblocked(Input(20))))

146

## [Day 21](http://adventofcode.com/2016/day/21): Scrambled Letters and Hash

We're attempting to scramble a password given a set of instructions which scramble the input.

### Operations
- `swap position X with position Y` means that the letters at indexes `X` and `Y` (counting from 0) should be swapped.
- `swap letter X with letter Y` means that the letters `X` and `Y` should be swapped (regardless of where they appear in the string).
- `rotate left/right X steps` means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
- `rotate based on position of letter X` means that the whole string should be rotated to the right based on the index of letter X (counting from 0) as determined before this instruction does any rotations. Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least 4.
- `reverse positions X through Y` means that the span of letters at indexes X through Y (including the letters at X and Y) should be reversed in order.
- `move position X to position Y` means that the letter which is at index X should be removed from the string, then inserted such that it ends up at index Y.

As these are individually simple lets encode these rules in some assertions then work our way through them.

In [7]:
def swap(text, a, b):
    if isinstance(a, int) or a.isdigit():
        return swap_pos(text, int(a), int(b))
    return swap_let(text, a, b)
        
def swap_pos(text, a, b):
    if a > b: a, b = b, a
    return text[:a] + text[b] + text[a+1:b] + text[a] + text[b+1:]

def swap_let(text, a, b):
    return swap_pos(text, text.index(a), text.index(b))

def reverse(text, a, b):
    if a > b: a, b = b, a
    if a == 0: return text[:a] + text[b::-1] + text[b+1:]
    return text[:a] + text[b:a-1:-1] + text[b+1:]

def rotate(text, a):
    if isinstance(a, int) or a.isdigit():
        return rotate_pos(text, int(a))
    return rotate_let(text, a)

def rotate_pos(text, rot):
    """
    Rotate the text rot indexes.
    When `rot` is positive, rotate left, when `rot` is negative, rotate right.
    """
    return text[rot%len(text):] + text[:rot%len(text)] 

def move(text, a, b):
    if a < b: return text[:a] + text[a+1:b+1] + text[a] + text[b+1:]
    else: return text[:b] + text[a] + text[b:a] + text[a+1:]
    
def rotate_let(text, let):
    rotations = 1 + text.index(let)
    if rotations >= 4: rotations += 1
    return rotate_pos(text, -rotations)

In [8]:
assert swap('abcde', 4, 0) == 'ebcda'
assert swap('ebcda', 'd', 'b') == 'edcba'
assert reverse('edcba', 0, 4) == 'abcde'
assert rotate('abcde', 1) == 'bcdea'
assert move('bcdea', 1, 4) == 'bdeac'
assert move('bdeac', 3, 0) == 'abdec'
assert rotate('abcde', 1) == 'bcdea'
assert rotate('abdec', 'b') == 'ecabd'

In [9]:
def parse22(input):
    words = input.split()
    command = ' '.join(words[0:2])
    if   command == 'swap letter': return swap(password, words[2], words[5])
    elif command == 'move position': return move(password, int(words[2]), int(words[5]))
    elif command == 'reverse positions': return reverse(password, int(words[2]), int(words[4]))
    elif command == 'rotate based': return rotate(password, words[6])
    elif command == 'swap position': return swap(password, words[2], words[5])
    elif command == 'rotate left': return rotate(password, int(words[2]))
    elif command == 'rotate right': return rotate(password, -int(words[2]))    

In [10]:
password = 'abcdefgh'
for line in Input(21):
     password = parse22(line)
password

'fdbgache'